In [34]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

from data import (TradeCapacityMapFacilitator, StackedQuantityEvolutionFacilitator, 
                  StackedQuantityEvolutionFacilitatorBase, HourlyTechActivityRateFacilitator,
                  ProductionByTechnologyForFuelStackedQuantityEvolutionFacilitator,
                  StorageDischargeHourlyRateFacilitator, AnnualUseStackedQuantityEvolutionFacilitator)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
CURRENT_DIR = Path().resolve()
SOLUTION_FILEPATH = CURRENT_DIR / "season_vs_daily_data" / "solution_season_data_daily.sol"
TITLE_ADDON = " - Season"
WRITE_IMAGE = False

### Trade capacities between European countries 2018 and 2050

In [36]:
trade_capacity_plotter = TradeCapacityMapFacilitator(sol_path=SOLUTION_FILEPATH, year=2018)

relevant_data = trade_capacity_plotter.get_relevant_data()

relevant_data.TotalTradeCapacity.filter_by_identifier(column="Fuel", identifier="Power")

fig = trade_capacity_plotter.plot(relevant_data)

fig.layout.title = "Power Trade Capacity in 2018" + TITLE_ADDON

fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))

if WRITE_IMAGE:
    fig.write_image("PowerTradeCapacity2018" + TITLE_ADDON.replace(" ", "") + ".png")

fig.show()

No unit conversion applied!


In [37]:
trade_capacity_plotter = TradeCapacityMapFacilitator(sol_path=SOLUTION_FILEPATH, year=2050)

relevant_data = trade_capacity_plotter.get_relevant_data()

relevant_data.TotalTradeCapacity.filter_by_identifier(column="Fuel", identifier="Power")

fig = trade_capacity_plotter.plot(relevant_data)

fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))

fig.layout.title = "Power Trade Capacity in 2050" + TITLE_ADDON

if WRITE_IMAGE:
    fig.write_image("PowerTradeCapacity2050.pdf" + TITLE_ADDON.replace(" ", "") + ".png")

fig.show()

No unit conversion applied!


### Storage capacities

In [38]:
storage_capacity_plotter = StackedQuantityEvolutionFacilitator(sol_path=SOLUTION_FILEPATH, region="DE")

relevant_data = storage_capacity_plotter.get_relevant_data()
#relevant_data.filter_by_containing_string(column="Technology", identifier="D_")
#relevant_data.filter_by_list(column="Technology", by_filter=["D_Heat_HLDH", "D_Heat_HLI", "D_Heat_HLB", 
#                                                             "D_Battery_Li-Ion", "D_Battery_Redox", "D_Heat_HLR"])
relevant_data.filter_by_containing_string(column="Technology", identifier="D_")


fig = storage_capacity_plotter.plot(relevant_data)

fig.update_layout(
    width=1400,
    height=800,
    title="Storage Capacity Germany" + TITLE_ADDON,
    yaxis_title="GW",
    xaxis_title="Year",
)

if WRITE_IMAGE:
    fig.write_image("StorageProductionGermany.pdf" + TITLE_ADDON.replace(" ", "") + ".png")

fig.show()

No unit conversion applied!


In [39]:
storage_capacity_plotter = StackedQuantityEvolutionFacilitator(sol_path=SOLUTION_FILEPATH, region="All")

relevant_data = storage_capacity_plotter.get_relevant_data()
#relevant_data.filter_by_containing_string(column="Technology", identifier="D_")
#relevant_data.filter_by_list(column="Technology", by_filter=["D_Heat_HLDH", "D_Heat_HLI", "D_Heat_HLB", 
#                                                             "D_Battery_Li-Ion", "D_Battery_Redox", "D_Heat_HLR"])
relevant_data.filter_by_containing_string(column="Technology", identifier="D_")
relevant_data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name="All", column_to_sum="Value")

fig = storage_capacity_plotter.plot(relevant_data)

fig.update_layout(
    width=1400,
    height=800,
    title="Storage Capacity All Regions" + TITLE_ADDON,
    yaxis_title="GW",
    xaxis_title="Year",
)

#if WRITE_IMAGE:
fig.write_image("StorageProductionAllRegions.pdf" + TITLE_ADDON.replace(" ", "") + ".png")

fig.show()

No unit conversion applied!


### Hourly data

In [40]:
tech_activity_hourly_facilitation = HourlyTechActivityRateFacilitator(sol_path=SOLUTION_FILEPATH, region="All", year=2050)

relevant_data = tech_activity_hourly_facilitation.get_relevant_data()

relevant_data.filter_by_list(column="Technology", by_filter=["D_Heat_HLDH", "D_Heat_HLI", "D_Heat_HLB"])
relevant_data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name="All", column_to_sum="Value")

fig = tech_activity_hourly_facilitation.plot(relevant_data)

fig.update_layout(title="Hourly Storage Tech Activity All Regions" + TITLE_ADDON, height=800, width=1400, font=dict(size=16))

if WRITE_IMAGE:
    fig.write_image("HourlyTechActivityAllRegions.pdf" + TITLE_ADDON.replace(" ", "") + ".png")

fig.show()

Converting from PetaJoules to TerraWattHours


In [41]:
tech_activity_hourly_facilitation = StorageDischargeHourlyRateFacilitator(sol_path=SOLUTION_FILEPATH, region="All", year=2050)

relevant_data = tech_activity_hourly_facilitation.get_relevant_data()
relevant_data.filter_by_containing_string_in_list(column="Technology", identifier_list=["D_Heat_HLDH", "D_Heat_HLI", "D_Heat_HLB"])
relevant_data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name="All", column_to_sum="Value")


# Filter out those who are 0 for all hours
techs_over_zero = relevant_data.df[relevant_data.df["Value"] != 0.0]["Technology"].unique()
relevant_data.filter_by_list(column="Technology", by_filter=techs_over_zero)

fig = tech_activity_hourly_facilitation.plot(relevant_data)

fig.update_layout(title="Hourly Storage Tech Activity All Regions" + TITLE_ADDON, height=800, width=1400, font=dict(size=16))

fig.write_image("HourlyTechActivityAllRegions.pdf" + TITLE_ADDON.replace(" ", "") + ".png")

fig.show()

Converting from PetaJoules to TerraWattHours


### Production of fuel by technology

In [45]:
region = "All"

#production_fuel = ProductionByTechnologyForFuelStackedQuantityEvolutionFacilitator(SOLUTION_FILEPATH, region=region,
#                                                                                   fuels=["Heat_Low_Building"])

production_fuel = ProductionByTechnologyForFuelStackedQuantityEvolutionFacilitator(SOLUTION_FILEPATH, region=region,
                                                                                   fuels=['Heat_Low_Residential', 'Heat_Low_Industrial', 'Heat_Medium_Industrial', 
                                                                                          'Heat_High_Industrial', 'Cool_Low_Building', 'Heat_District', 'Heat_Low_Building'])

relevant_data = production_fuel.get_relevant_data()

if region == "All":
    relevant_data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name="All", column_to_sum="Value")

# Keep techs not containing "D_"
relevant_data.df = relevant_data.df[~relevant_data.df["Technology"].str.contains("D_")]

# If "Value" is bigger than zero for a specific tech combined with a fuel for any year, keep all
tech_and_fuels_over_zero = relevant_data.df[relevant_data.df["Value"] > 0][["Technology", "Fuel"]].drop_duplicates()

relevant_data.df = relevant_data.df.merge(tech_and_fuels_over_zero, on=["Technology", "Fuel"], how="inner")

fig = production_fuel.plot(relevant_data)
fig.update_layout(title=f"Production of Low District Heat, Region: {region}" + TITLE_ADDON)

fig.update_layout(width=1400, height=1100)

#if WRITE_IMAGE:
fig.write_image(f"ProductionLowDistrictHeat_{region}" + TITLE_ADDON.replace(" ", "") + ".png")

fig.show()

Converting from PetaJoules to TerraWattHours


# Total annual usage

In [43]:
annual_use_facilitator = AnnualUseStackedQuantityEvolutionFacilitator(SOLUTION_FILEPATH, region="All")

relevant_data = annual_use_facilitator.get_relevant_data()
relevant_data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name="All", column_to_sum="Value")

fig = annual_use_facilitator.plot(relevant_data)

fig.update_layout(width=1400, height=1100)
fig.show()

Converting from PetaJoules to TerraWattHours
